In [3]:
import torch
import attr
from pathlib import Path
from torchvision import datasets, transforms
import torchvision
import matplotlib.pyplot as plt
from sklearn import metrics as sk_metrics
import numpy as np

In [4]:
@attr.s(auto_attribs=True)
class DataPreparation:
    data_dir: Path
    device: str = attr.ib(default=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"), init=False)

    @staticmethod
    def data_transformations():
        data_transforms = {
            'test': transforms.Compose([
                transforms.CenterCrop((200,1500)),
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        }
        return data_transforms
    
    def create_dataloaders(self, batch_size, shuffle, num_workers):
        data_transforms = self.data_transformations()
        
        image_datasets = {
            'test': datasets.ImageFolder(self.data_dir, data_transforms['test'])
        }
        dataloaders = {
            'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
        }
        dataset_sizes = {
            'test': len(image_datasets['test'])
        }
        return dataloaders, dataset_sizes

In [5]:
variants = ["final", "mid", "initial",  "multiclass"]
models = ["resnet50_d_02_t_20_10_final_beat", "resnet50_d_02_t_22_52_mid_beat", "resnet50_d_02_t_20_28_initial_beat", "resnet50_d_05_t_00_40"]

In [6]:
index = 0

In [8]:
data_prep = DataPreparation(f'../data/figures_{variants[index]}/test')
data, size = data_prep.create_dataloaders(16, False, 4)

In [9]:
model_path = Path().cwd().parents[0] / f"models/{models[index]}.pth" 
model = torch.load(model_path)
model = model.eval()

In [10]:
y_pred = []
y_true = []

for inputs, labels in data['test']:
    inputs = inputs.to(0)
    labels = labels.to(0)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    y_pred.append(preds.cpu().numpy())
    y_true.append(labels.cpu().numpy())


RuntimeError: CUDA out of memory. Tried to allocate 14.00 MiB (GPU 0; 7.80 GiB total capacity; 1.12 GiB already allocated; 8.31 MiB free; 1.17 GiB reserved in total by PyTorch)

In [ ]:
y_pred = np.concatenate(y_pred)
y_true = np.concatenate(y_true)

In [ ]:
acc = sk_metrics.accuracy_score(y_pred, y_true)
f1 = sk_metrics.f1_score(y_pred, y_true, average="weighted")
pre = sk_metrics.precision_score(y_pred, y_true, average="weighted")

In [22]:
acc, f1, pre

(0.9596497906357061, 0.9561070488332196, 0.9578081046604476)